In [2]:
#libraries
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.preprocessing import normalize

In [3]:
#read the data & have randome sample for reproducability
recipes = pd.read_csv('../data/full_clean_data.csv')

In [4]:
##rename the dataframe index
recipes.index.names = ['recipes_id']
recipes.head(2)

,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed
recipes_id,,,,,,,,,,,,,,
0,Dessert Crepes,https://www.allrecipes.com/recipe/19037/desser...,breakfast-and-brunch,4.80,1156,"['4 eggs, lightly beaten ', ' 1\u2009⅓ cups m...",20,8,8 crepes,163.8,17.2,7.7,6.4,"['milk', 'flour', 'sugar']"
1,Chicken Parmesan,https://www.allrecipes.com/recipe/223042/chick...,world-cuisine,4.83,4245,"['4 skinless, boneless chicken breast halves ...",60,4,4 servings,470.8,24.8,24.9,42.1,"['chicken breast', 'egg', 'panko bread', 'parm..."


In [5]:
##normalize the numerical column (total+nutritional values) in new dataframe << because i dont want to normalize all other features
newrecipes = recipes[['total', 'calories', 'carbohydrates_g', 'fat_g', 'protein_g']]
recipes_norm = pd.DataFrame(normalize(newrecipes, axis=0))
recipes_norm.columns = newrecipes.columns
recipes_norm.index = newrecipes.index
recipes_norm.head(2)

,total,calories,carbohydrates_g,fat_g,protein_g
recipes_id,,,,,
0,0.000837,0.003888,0.003539,0.003818,0.002726
1,0.002512,0.011174,0.005103,0.012346,0.017931


In [6]:
##convert index to column
recipes['recipes_id'] = recipes.index


In [7]:

#####creare a function for selected recipe. Guidance reference https://www.kaggle.com/code/yyzz1010/content-based-filtering-recipe-recommender?scriptVersionId=21219821&cellId=22 
def selected_recipe(recipes_id):
    recipe_df = recipes.set_index('recipes_id') # set a new index to this DataFrame
    x = "{}  {}".format(recipes_id, recipe_df.at[recipes_id, 'name']) #placeholders 
    print(x)

In [8]:
#####creare a function that recommends top 3 similar recipe using cosine. Guidance reference https://www.kaggle.com/code/yyzz1010/content-based-filtering-recipe-recommender?scriptVersionId=21219821&cellId=23
def recipe_recommender(recipes_id):
    
     allRecipes = pd.DataFrame(recipes_norm.index)
     allRecipes = allRecipes[allRecipes.recipes_id != recipes_id]
     allRecipes["distance"] = allRecipes["recipes_id"].apply(lambda x: cosine(recipes_norm.loc[recipes_id], recipes_norm.loc[x]))
     Top3NRecommendation = allRecipes.sort_values(["distance"]).head(3).sort_values(by=['distance', 'recipes_id'])
     # sort by distance then recipe id, the smaller value of recipe id will be picked. 
    
     recipe_df = recipes.set_index('recipes_id')
     recipes_id = [recipes_id]
     recipe_list = []
     for recipeid in Top3NRecommendation.recipes_id:
        recipes_id.append(recipeid)   # list of recipe id of selected recipe and recommended recipe
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'url']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'name']))
     return recipe_list

In [12]:
##get the name of the selected recipe based on id
selected_recipe(12521)

12521  White Bean Tabbouleh


In [11]:
#recommend the top 3 most similar (total, calories, protein, carbs, fat)
recipe_recommender(12521)

['10633  https://www.allrecipes.com/recipe/244326/greek-farro-salad/',
 '10633  Greek Farro Salad',
 '13949  https://www.allrecipes.com/recipe/218884/uptown-cowboy-caviar/',
 '13949  Uptown Cowboy Caviar',
 '10643  https://www.allrecipes.com/recipe/232590/roasted-beet-and-kale-salad/',
 '10643  Roasted Beet and Kale Salad']